In [5]:
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import os
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')


In [3]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [4]:
def extract_user_playlist(url):
    # Split the url and use Spotipy to retrieve the track information for each song in the playlist
    playlist_url = url.split("/")[4].split("?")[0]
    playlist_tracks = sp.playlist_tracks(playlist_url)

    # For loop to extract the necessary track information
    track_ids = []
    track_titles = []
    track_artist = []
    track_album_art = []
    track_album_url = []

    for track in playlist_tracks['items']:
        track_ids.append(track['track']['id'])
        track_titles.append(track['track']['name'])
        track_album_art.append(track['track']['album']['images'][2]['url'])
        track_album_url.append(track['track']['album']['external_urls']['spotify'])
        artist_list = []
        for artist in track['track']['artists']:
            artist_list.append(artist['name'])
        track_artist.append(artist_list[0])

    # Create a dataframe from the track_ids to bring in features
    features = sp.audio_features(track_ids)

    features_df = pd.DataFrame(data=features, columns=features[0].keys())
    features_df['title'] = track_titles
    features_df['artist'] = track_artist
    features_df['album_art'] = track_album_art
    features_df['album_url'] = track_album_url
    features_df = features_df[['artist','title','album_url','album_art','id','danceability','energy','loudness','speechiness','acousticness','liveness','valence']]
    
    return features_df
    

In [55]:
user_playlist = extract_user_playlist(
    'https://open.spotify.com/playlist/5DryjHx6o4EWrGvIX33rnq?si=a01b40825c8f402f')

user_playlist.head()


,artist,title,album_url,album_art,id,danceability,energy,loudness,speechiness,acousticness,liveness,valence
0,Gorillaz,Feel Good Inc.,https://open.spotify.com/album/0bUTHlWbkSQysoM...,https://i.scdn.co/image/ab67616d0000485119d85a...,0d28khcov6AiegSCpG5TuT,0.818,0.705,-6.679,0.1770,0.00836,0.613,0.772
1,Mac Miller,Weekend (feat. Miguel),https://open.spotify.com/album/2Tyx5dLhHYkx6ze...,https://i.scdn.co/image/ab67616d00004851ee0f38...,6GnhWMhgJb7uyiiPEiEkDA,0.843,0.435,-8.442,0.1780,0.47000,0.147,0.190
2,Mac Miller,Come Back to Earth,https://open.spotify.com/album/5wtE5aLX5r7jOos...,https://i.scdn.co/image/ab67616d00004851175c57...,01z2fBGB8Hl3Jd3zXe4IXR,0.272,0.238,-12.148,0.0349,0.89000,0.275,0.119
3,Hotel Ugly,Shut up My Moms Calling,https://open.spotify.com/album/4qVzNqB4cskFInY...,https://i.scdn.co/image/ab67616d000048514e9657...,3hxIUxnT27p5WcmjGUXNwx,0.485,0.409,-10.711,0.1010,0.32500,0.103,0.376
4,Kanye West,All Falls Down,https://open.spotify.com/album/3ff2p3LnR6V7m6B...,https://i.scdn.co/image/ab67616d000048512892dd...,2cYZpcIV39X48RnOFM7w2V,0.643,0.717,-4.876,0.5060,0.18400,0.152,0.460


In [48]:
for row in user_playlist.iterrows():
    print(
        f"{row[1]['artist']} - {row[1]['title']} - {row[1]['album_art']} - {row[1]['id']}")


Gorillaz - Feel Good Inc. - https://i.scdn.co/image/ab67616d0000485119d85a472f328a6ed9b704cf - 0d28khcov6AiegSCpG5TuT
Mac Miller - Weekend (feat. Miguel) - https://i.scdn.co/image/ab67616d00004851ee0f38410382a255e4fb15f4 - 6GnhWMhgJb7uyiiPEiEkDA
Mac Miller - Come Back to Earth - https://i.scdn.co/image/ab67616d00004851175c577a61aa13d4fb4b6534 - 01z2fBGB8Hl3Jd3zXe4IXR
Hotel Ugly - Shut up My Moms Calling - https://i.scdn.co/image/ab67616d000048514e9657f8a8d50fb54f4d2c94 - 3hxIUxnT27p5WcmjGUXNwx
Kanye West - All Falls Down - https://i.scdn.co/image/ab67616d000048512892dd14331eaa4cee20194d - 2cYZpcIV39X48RnOFM7w2V
Gorillaz - Clint Eastwood - https://i.scdn.co/image/ab67616d00004851f6c46838e4425ea96e2562fe - 1RKUoGiLEbcXN4GY4spQDx
Kid Cudi - Pursuit Of Happiness (Nightmare) - https://i.scdn.co/image/ab67616d00004851a487deeecb334b6619489d74 - 6MtKObWYda2qnNIpJI21uD
Marlon Craft - Cheap Date - https://i.scdn.co/image/ab67616d0000485194e8a584cd4aa81d493e7a1f - 045yTNzD1xgxtATyd81jCy
Blvck Svm

In [56]:
# Filtering through the data to see what we need to pull
playlist_tracks = sp.playlist_tracks(
    'https://open.spotify.com/playlist/5DryjHx6o4EWrGvIX33rnq?si=d566f40322a3430e')
playlist_tracks['items'][0]['track']['album']


{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3AA28KZvwAUcZuOKwyblJQ'},
   'href': 'https://api.spotify.com/v1/artists/3AA28KZvwAUcZuOKwyblJQ',
   'id': '3AA28KZvwAUcZuOKwyblJQ',
   'name': 'Gorillaz',
   'type': 'artist',
   'uri': 'spotify:artist:3AA28KZvwAUcZuOKwyblJQ'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'ET',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',
  'KG',
 

In [1]:
'https://open.spotify.com/track/0d28khcov6AiegSCpG5TuT'


'https://open.spotify.com/track/0d28khcov6AiegSCpG5TuT'

In [25]:
## Song Recommender

def song_chooser(url):
    user_df = extract_user_playlist(url)
    clean_df = user_df[['acousticness', 'danceability', 'energy','liveness', 'loudness', 'speechiness', 'valence', 'title']]
    user_avg_scores = clean_df.mean(axis=0)
    search_variable = user_avg_scores.idxmax()
    distance = []
    for index, row in clean_df.iterrows():
        point1 = user_avg_scores[search_variable]
        point2 = np.array(row[search_variable])
        dist = np.linalg.norm(point1 - point2)
        distance.append(dist)
    clean_df[f'distance_for_{search_variable}'] = distance
    clean_df.set_index('title', inplace= True)
    song_search = clean_df[f'distance_for_{search_variable}'].idxmin()
    
    return song_search

In [26]:
song_chooser('https://open.spotify.com/playlist/5DryjHx6o4EWrGvIX33rnq?si=d566f40322a3430e')

C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_16900\2050093041.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_avg_scores = clean_df.mean(axis=0)
C:\Users\dejans.THOMASPUMP\AppData\Local\Temp\ipykernel_16900\2050093041.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[f'distance_for_{search_variable}'] = distance


'Broadway Girls (feat. Morgan Wallen)'

In [27]:
user_df = extract_user_playlist('https://open.spotify.com/playlist/5DryjHx6o4EWrGvIX33rnq?si=d566f40322a3430e')
clean_df = user_df[['acousticness', 'danceability', 'energy','liveness', 'loudness', 'speechiness', 'valence', 'title']]
clean_df.set_index('title', inplace= True)
clean_df
user_avg_scores = clean_df.mean(axis=0)
search_variable = clean_df.idxmax()
distance = []
for index, row in clean_df.iterrows():
        point1 = user_avg_scores[search_variable]
        point2 = np.array(row[search_variable])
        dist = np.linalg.norm(point1 - point2)
        distance.append(dist)

KeyError: "None of [Index(['Superstar Sh*t', 'Too Comfortable', 'Willie Dynamite',\n       'Feel Good Inc.', 'Broadway Girls (feat. Morgan Wallen)',\n       'All Falls Down', 'I Ain't Worried'],\n      dtype='object')] are in the [index]"